In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/train-ranking/data/processed/public/task_1_query-product_ranking/train-v0.2.csv
/kaggle/input/product-catalogue/data/processed/public/task_1_query-product_ranking/product_catalogue-v0.2.csv
/kaggle/input/test-ranking-public/data/processed/public/task_1_query-product_ranking/test_public-v0.2.csv


In [2]:
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', 10)

In [3]:
!pip install datasets==2.1.0

In [4]:
from datasets import ClassLabel, Value, DatasetDict, Dataset

In [5]:
from transformers import AutoModelForMaskedLM, AutoTokenizer
checkpoint = "distilbert-base-multilingual-cased"
model = AutoModelForMaskedLM.from_pretrained(checkpoint)
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

Downloading:   0%|          | 0.00/466 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/517M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/972k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.87M [00:00<?, ?B/s]

In [6]:
prod_cat = pd.read_csv('../input/product-catalogue/data/processed/public/task_1_query-product_ranking/product_catalogue-v0.2.csv')

In [7]:
prod_cat.head()

,product_id,product_title,product_description,product_bullet_point,product_brand,product_color_name,product_locale
0,B0188A3QRM,"Amazon Basics Woodcased #2 Pencils, Unsharpened, HB Lead - Box of 144, Bulk Box",NaN,"144 woodcase #2 HB pencils made from high-quality wood for clean, easy sharpening\nStrong medium-soft lead produces long-lasting, smooth, readable strokes\nRounded hexagonal shape with satin-smooth finish for a secure, comfortable grip\nSoft, smudge-free, latex-free eraser secured to the end for conveniently wiping away mistakes",Amazon Basics,Yellow,us
1,B075VXJ9VG,"BAZIC Pencil #2 HB Pencils, Latex Free Eraser, Wood Free Yellow Unsharpened Pencils for Exam School Office (12/Pack), 1-Pack","<p><strong>BACK TO BAZIC</strong></p><p>Our goal is to provide each customer with long-lasting supplies at an affordable cost. Since 1998, we’ve delivered on this promise and will only continue to improve every year. We’ve built our brand on integrity and quality, so customers know exactly what to expect.</p> <p><strong>COMMITTED TO VALUES</strong></p><p>We are a value-driven company, guided by the principles of excellence through strong product design at low cost. Our commitment to these values is reflected in our dedication to improving current products and developing new exciting products for our consumers.</p> <p><strong>FOCUSED ON OUR MISSION</strong></p><p>Our currency is ideas. We thrive on imagination, passion and leadership. We have great products and will to continue to rise with our customer expectations.</p> <p><strong>SUCCESS BASED ON SATISFACTION</strong></p><p>Each and every product we send out, we expect our 100% customer satisfaction. While our supplies are sourced from all across the world, our success stems from individual consumer fulfillment. We create products that people want to recommend to others.</p>","&#11088; UN-SHARPENED #2 PREMIUM PENCILS. Each pack comes with 12 un-sharpened pencils. Unlike most others out there, these allow kids the simple joy of sharpening a brand new pencil for the very 1st time!\n&#11088; CLASSIC BARREL. Made from premium-quality wood pencil is bonded with a break-resistant number two core for smooth writing as easily erased. Hexagonal shape prevents rolling and comfortable to hold.\n&#11088; QUALITY. Pencils are #2 HB for general writing, drawing and sketching. Pencils glide smoothly over paper and is highly break resistant. Pencils include a smudge-resistant eraser top for convenience.\n&#11088; LATEX-FREE ERASER. The smudge-less eraser is latex-free, which provides enough toughness for heavy erasures. These smudge-free erasers wipe the page clean, making corrections easy and fast.\n&#11088; WIDE APPLICATION. Suitable for all general writing applications such as notes, letters, and you can use it for office, classroom, home and other occasions.",BAZIC Products,12-count,us
2,B07G7F6JZ6,"Emraw Pre Sharpened Round Primary Size No 2 Jumbo Pencils for Preschoolers, Elementary Kids - Pack of 8 Premium Fat Pencils","<p><b>Emraw Pre-Sharpened #2 HB Wood Pencils - Pack of 8 Pre-Schoolers Pencils</b></p><p>Looking for pencils for kid, office or every day general purpose repetitive use? Tired of pencils that do not sharpen well? Disappointed having broken leads repeatedly? Or, looking for pencils that require less sharpening and lasts longer?</p><p>Search no further, Emraw num 2 jumbo pencils pack have you covered. You just found the best deal. You will love them. Just the right size for pre-kindergarten toddlers. Perfect for elderly hands having difficulty grasping smaller items. Each pencil is pre-sharpened so you can literally pick straight from the box and start using. HB (no2) hardness grading with thicker leads make them perfect for most types of writing or sketching.</p><p>Made of high quality non-toxic cedar wood so they sharpen nicely with any quality dual-holed hand sharpener or adjustable electric sharpener, (not compatible with standard pencil sharpener). Rounded circular sh

In [8]:
for locale, _df in prod_cat.groupby(['product_locale']):
    print(locale, _df.product_id.is_unique)
    assert _df.product_id.is_unique

es True
jp True
us True


In [9]:
def get_product_input(row):
    product_columns = ['product_title', 'product_locale', 'product_description', 'product_bullet_point']
    product_info = ''
    for col in product_columns:
        product_info += str(row.get(col, '')) + ' '
    return product_info.strip()

d = {'product_title': 'hi hello', 'product_locale': 'en', 'product_description': 'nike', 'product_bullet_point': 'shoes'}
assert get_product_input(d) == 'hi hello en nike shoes'

In [10]:
prod_cat.fillna('', inplace=True)
prod_cat['product_input'] = prod_cat.apply(lambda row: get_product_input(row), axis=1)

In [11]:
prod_cat.head(4)

,product_id,product_title,product_description,product_bullet_point,product_brand,product_color_name,product_locale,product_input
0,B0188A3QRM,"Amazon Basics Woodcased #2 Pencils, Unsharpened, HB Lead - Box of 144, Bulk Box",,"144 woodcase #2 HB pencils made from high-quality wood for clean, easy sharpening\nStrong medium-soft lead produces long-lasting, smooth, readable strokes\nRounded hexagonal shape with satin-smooth finish for a secure, comfortable grip\nSoft, smudge-free, latex-free eraser secured to the end for conveniently wiping away mistakes",Amazon Basics,Yellow,us,"Amazon Basics Woodcased #2 Pencils, Unsharpened, HB Lead - Box of 144, Bulk Box us 144 woodcase #2 HB pencils made from high-quality wood for clean, easy sharpening\nStrong medium-soft lead produces long-lasting, smooth, readable strokes\nRounded hexagonal shape with satin-smooth finish for a secure, comfortable grip\nSoft, smudge-free, latex-free eraser secured to the end for conveniently wiping away mistakes"
1,B075VXJ9VG,"BAZIC Pencil #2 HB Pencils, Latex Free Eraser, Wood Free Yellow Unsharpened Pencils for Exam School Office (12/Pack), 1-Pack","<p><strong>BACK TO BAZIC</strong></p><p>Our goal is to provide each customer with long-lasting supplies at an affordable cost. Since 1998, we’ve delivered on this promise and will only continue to improve every year. We’ve built our brand on integrity and quality, so customers know exactly what to expect.</p> <p><strong>COMMITTED TO VALUES</strong></p><p>We are a value-driven company, guided by the principles of excellence through strong product design at low cost. Our commitment to these values is reflected in our dedication to improving current products and developing new exciting products for our consumers.</p> <p><strong>FOCUSED ON OUR MISSION</strong></p><p>Our currency is ideas. We thrive on imagination, passion and leadership. We have great products and will to continue to rise with our customer expectations.</p> <p><strong>SUCCESS BASED ON SATISFACTION</strong></p><p>Each and every product we send out, we expect our 100% customer satisfaction. While our supplies are sourced from all across the world, our success stems from individual consumer fulfillment. We create products that people want to recommend to others.</p>","&#11088; UN-SHARPENED #2 PREMIUM PENCILS. Each pack comes with 12 un-sharpened pencils. Unlike most others out there, these allow kids the simple joy of sharpening a brand new pencil for the very 1st time!\n&#11088; CLASSIC BARREL. Made from premium-quality wood pencil is bonded with a break-resistant number two core for smooth writing as easily erased. Hexagonal shape prevents rolling and comfortable to hold.\n&#11088; QUALITY. Pencils are #2 HB for general writing, drawing and sketching. Pencils glide smoothly over paper and is highly break resistant. Pencils include a smudge-resistant eraser top for convenience.\n&#11088; LATEX-FREE ERASER. The smudge-less eraser is latex-free, which provides enough toughness for heavy erasures. These smudge-free erasers wipe the page clean, making corrections easy and fast.\n&#11088; WIDE APPLICATION. Suitable for all general writing applications such as notes, letters, and you can use it for office, classroom, home and other occasions.",BAZIC Products,12-count,us,"BAZIC Pencil #2 HB Pencils, Latex Free Eraser, Wood Free Yellow Unsharpened Pencils for Exam School Office (12/Pack), 1-Pack us <p><strong>BACK TO BAZIC</strong></p><p>Our goal is to provide each customer with long-lasting supplies at an affordable cost. Since 1998, we’ve delivered on this promise and will only continue to improve every year. We’ve built our brand on integrity and quality, so customers know exactly what to expect.</p> <p><strong>COMMITTED TO VALUES</strong></p><p>We are a value-driven company, guided by the principles of excellence through strong product design at low cost. Our commitment to these values is reflected in our dedication to improving current products and dev

In [12]:
escapes = ''.join([chr(char) for char in range(1, 32)])
translator = str.maketrans('', '', escapes)
prod_cat['product_input'] = prod_cat['product_input'].apply(lambda inp: inp.translate(translator))

In [13]:
prod_cat.head()

,product_id,product_title,product_description,product_bullet_point,product_brand,product_color_name,product_locale,product_input
0,B0188A3QRM,"Amazon Basics Woodcased #2 Pencils, Unsharpened, HB Lead - Box of 144, Bulk Box",,"144 woodcase #2 HB pencils made from high-quality wood for clean, easy sharpening\nStrong medium-soft lead produces long-lasting, smooth, readable strokes\nRounded hexagonal shape with satin-smooth finish for a secure, comfortable grip\nSoft, smudge-free, latex-free eraser secured to the end for conveniently wiping away mistakes",Amazon Basics,Yellow,us,"Amazon Basics Woodcased #2 Pencils, Unsharpened, HB Lead - Box of 144, Bulk Box us 144 woodcase #2 HB pencils made from high-quality wood for clean, easy sharpeningStrong medium-soft lead produces long-lasting, smooth, readable strokesRounded hexagonal shape with satin-smooth finish for a secure, comfortable gripSoft, smudge-free, latex-free eraser secured to the end for conveniently wiping away mistakes"
1,B075VXJ9VG,"BAZIC Pencil #2 HB Pencils, Latex Free Eraser, Wood Free Yellow Unsharpened Pencils for Exam School Office (12/Pack), 1-Pack","<p><strong>BACK TO BAZIC</strong></p><p>Our goal is to provide each customer with long-lasting supplies at an affordable cost. Since 1998, we’ve delivered on this promise and will only continue to improve every year. We’ve built our brand on integrity and quality, so customers know exactly what to expect.</p> <p><strong>COMMITTED TO VALUES</strong></p><p>We are a value-driven company, guided by the principles of excellence through strong product design at low cost. Our commitment to these values is reflected in our dedication to improving current products and developing new exciting products for our consumers.</p> <p><strong>FOCUSED ON OUR MISSION</strong></p><p>Our currency is ideas. We thrive on imagination, passion and leadership. We have great products and will to continue to rise with our customer expectations.</p> <p><strong>SUCCESS BASED ON SATISFACTION</strong></p><p>Each and every product we send out, we expect our 100% customer satisfaction. While our supplies are sourced from all across the world, our success stems from individual consumer fulfillment. We create products that people want to recommend to others.</p>","&#11088; UN-SHARPENED #2 PREMIUM PENCILS. Each pack comes with 12 un-sharpened pencils. Unlike most others out there, these allow kids the simple joy of sharpening a brand new pencil for the very 1st time!\n&#11088; CLASSIC BARREL. Made from premium-quality wood pencil is bonded with a break-resistant number two core for smooth writing as easily erased. Hexagonal shape prevents rolling and comfortable to hold.\n&#11088; QUALITY. Pencils are #2 HB for general writing, drawing and sketching. Pencils glide smoothly over paper and is highly break resistant. Pencils include a smudge-resistant eraser top for convenience.\n&#11088; LATEX-FREE ERASER. The smudge-less eraser is latex-free, which provides enough toughness for heavy erasures. These smudge-free erasers wipe the page clean, making corrections easy and fast.\n&#11088; WIDE APPLICATION. Suitable for all general writing applications such as notes, letters, and you can use it for office, classroom, home and other occasions.",BAZIC Products,12-count,us,"BAZIC Pencil #2 HB Pencils, Latex Free Eraser, Wood Free Yellow Unsharpened Pencils for Exam School Office (12/Pack), 1-Pack us <p><strong>BACK TO BAZIC</strong></p><p>Our goal is to provide each customer with long-lasting supplies at an affordable cost. Since 1998, we’ve delivered on this promise and will only continue to improve every year. We’ve built our brand on integrity and quality, so customers know exactly what to expect.</p> <p><strong>COMMITTED TO VALUES</strong></p><p>We are a value-driven company, guided by the principles of excellence through strong product design at low cost. Our commitment to these values is reflected in our dedication to improving current products and developin

In [14]:
import re
# as per recommendation from @freylis, compile once only
CLEANR = re.compile('<.*?>')
def cleanhtml(raw_html):
  cleantext = re.sub(CLEANR, '', raw_html)
  return cleantext

prod_cat['product_input'] = prod_cat['product_input'].apply(lambda inp: cleanhtml(inp))

In [15]:
prod_cat.head()

,product_id,product_title,product_description,product_bullet_point,product_brand,product_color_name,product_locale,product_input
0,B0188A3QRM,"Amazon Basics Woodcased #2 Pencils, Unsharpened, HB Lead - Box of 144, Bulk Box",,"144 woodcase #2 HB pencils made from high-quality wood for clean, easy sharpening\nStrong medium-soft lead produces long-lasting, smooth, readable strokes\nRounded hexagonal shape with satin-smooth finish for a secure, comfortable grip\nSoft, smudge-free, latex-free eraser secured to the end for conveniently wiping away mistakes",Amazon Basics,Yellow,us,"Amazon Basics Woodcased #2 Pencils, Unsharpened, HB Lead - Box of 144, Bulk Box us 144 woodcase #2 HB pencils made from high-quality wood for clean, easy sharpeningStrong medium-soft lead produces long-lasting, smooth, readable strokesRounded hexagonal shape with satin-smooth finish for a secure, comfortable gripSoft, smudge-free, latex-free eraser secured to the end for conveniently wiping away mistakes"
1,B075VXJ9VG,"BAZIC Pencil #2 HB Pencils, Latex Free Eraser, Wood Free Yellow Unsharpened Pencils for Exam School Office (12/Pack), 1-Pack","<p><strong>BACK TO BAZIC</strong></p><p>Our goal is to provide each customer with long-lasting supplies at an affordable cost. Since 1998, we’ve delivered on this promise and will only continue to improve every year. We’ve built our brand on integrity and quality, so customers know exactly what to expect.</p> <p><strong>COMMITTED TO VALUES</strong></p><p>We are a value-driven company, guided by the principles of excellence through strong product design at low cost. Our commitment to these values is reflected in our dedication to improving current products and developing new exciting products for our consumers.</p> <p><strong>FOCUSED ON OUR MISSION</strong></p><p>Our currency is ideas. We thrive on imagination, passion and leadership. We have great products and will to continue to rise with our customer expectations.</p> <p><strong>SUCCESS BASED ON SATISFACTION</strong></p><p>Each and every product we send out, we expect our 100% customer satisfaction. While our supplies are sourced from all across the world, our success stems from individual consumer fulfillment. We create products that people want to recommend to others.</p>","&#11088; UN-SHARPENED #2 PREMIUM PENCILS. Each pack comes with 12 un-sharpened pencils. Unlike most others out there, these allow kids the simple joy of sharpening a brand new pencil for the very 1st time!\n&#11088; CLASSIC BARREL. Made from premium-quality wood pencil is bonded with a break-resistant number two core for smooth writing as easily erased. Hexagonal shape prevents rolling and comfortable to hold.\n&#11088; QUALITY. Pencils are #2 HB for general writing, drawing and sketching. Pencils glide smoothly over paper and is highly break resistant. Pencils include a smudge-resistant eraser top for convenience.\n&#11088; LATEX-FREE ERASER. The smudge-less eraser is latex-free, which provides enough toughness for heavy erasures. These smudge-free erasers wipe the page clean, making corrections easy and fast.\n&#11088; WIDE APPLICATION. Suitable for all general writing applications such as notes, letters, and you can use it for office, classroom, home and other occasions.",BAZIC Products,12-count,us,"BAZIC Pencil #2 HB Pencils, Latex Free Eraser, Wood Free Yellow Unsharpened Pencils for Exam School Office (12/Pack), 1-Pack us BACK TO BAZICOur goal is to provide each customer with long-lasting supplies at an affordable cost. Since 1998, we’ve delivered on this promise and will only continue to improve every year. We’ve built our brand on integrity and quality, so customers know exactly what to expect. COMMITTED TO VALUESWe are a value-driven company, guided by the principles of excellence through strong product design at low cost. Our commitment to these values is reflected in our dedication to improving current products and developing new exciting products for our consumers. FOCUSED ON OUR 

In [16]:
from sklearn.model_selection import train_test_split
train = []
test = []
for local, _df in prod_cat.groupby(['product_locale']):
    train_df, test_df = train_test_split(_df, test_size=0.3, random_state=9)
    train.append(train_df)
    test.append(test_df)

In [17]:
train_df = pd.concat(train, ignore_index=True)
test_df = pd.concat(test, ignore_index=True)

In [18]:
train_df.shape, test_df.shape

((618706, 8), (265162, 8))

In [19]:
train_df = train_df[['product_input']].reset_index(drop=True)
test_df = test_df[['product_input']].reset_index(drop=True)

In [20]:
train_df.shape, test_df.shape

((618706, 1), (265162, 1))

In [21]:
dataset = DatasetDict({
        'train': Dataset.from_pandas(train_df),
        'test': Dataset.from_pandas(test_df),
    }   
)

In [22]:
# Data collator used for language modeling. 
# Inputs are dynamically padded to the maximum length of a batch 
# if they are not all of the same length.

In [23]:
def tokenize(batch):
    # truncate to max_length if length of input > max_length
    return tokenizer(batch['product_input'], truncation=True, max_length=128, return_special_tokens_mask=True)

In [24]:
#dataset['train'] = dataset['train'].select([1,2,3,4,5,6,7,8,9,10])
#dataset['test'] = dataset['test'].select([1,2,3,4,5,6,7,8,9,10])
ds_mlm = dataset.map(tokenize, batched=True, batch_size=4)

  0%|          | 0/154677 [00:00<?, ?ba/s]

  0%|          | 0/66291 [00:00<?, ?ba/s]

In [25]:
ds_mlm = ds_mlm.remove_columns(['product_input'])

In [26]:
ds_mlm

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'special_tokens_mask'],
        num_rows: 618706
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'special_tokens_mask'],
        num_rows: 265162
    })
})

In [27]:
from transformers import DataCollatorForLanguageModeling, set_seed

In [28]:
set_seed(3)
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer,mlm_probability=0.15)
data_collator.return_tensors = "pt"

In [29]:
import torch
from transformers import AutoModelForMaskedLM
from transformers import Trainer
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir = "./", 
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    evaluation_strategy="epoch", 
    save_strategy="epoch",
    num_train_epochs=3,
    log_level="info", 
    report_to="none",
    save_total_limit=1,
    fp16=torch.cuda.is_available(),
    load_best_model_at_end=True
)


In [30]:
trainer = Trainer(
    model=model,
    tokenizer=tokenizer,
    args=training_args,
    data_collator=data_collator,
    train_dataset=ds_mlm['train'],
    eval_dataset=ds_mlm['test'],
)

Using amp half precision backend


In [31]:
import traceback
try:
    trainer.train()
except:
    print(traceback.format_exc())
    print("Training stopped")

The following columns in the training set  don't have a corresponding argument in `DistilBertForMaskedLM.forward` and have been ignored: special_tokens_mask. If special_tokens_mask are not expected by `DistilBertForMaskedLM.forward`,  you can safely ignore this message.


/opt/conda/lib/python3.7/site-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****


  Num examples = 618706


  Num Epochs = 3


  Instantaneous batch size per device = 4


...

Saving model checkpoint to ./checkpoint-309354


Configuration saved in ./checkpoint-309354/config.json


Model weights saved in ./checkpoint-309354/pytorch_model.bin


tokenizer config file saved in ./checkpoint-309354/tokenizer_config.json


Special tokens file saved in ./checkpoint-309354/special_tokens_map.json


In [ ]:
try:
    trainer.save_model('/kaggle/working/')
except:
    print("Trainer save stopped")